In [7]:
import os
from tqdm import tqdm
from huggingface_hub import login
import torch
import torch.nn as nn
import math
import einops
from fancy_einsum import einsum
import tqdm.auto as tqdm
import plotly.express as px
from jaxtyping import Float
from functools import partial
import transformer_lens.utils as utils
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from transformer_lens.hook_points import (
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, FactoredMatrix
from collections import defaultdict

In [8]:
with open("access.tok", "r") as file:
    access_token = file.read()
    login(token=access_token)

if torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Device: {device}")

Device: mps


In [9]:
from datasets import load_dataset  
from transformer_lens import HookedTransformer
from sae_lens import SAE, HookedSAETransformer

torch.set_grad_enabled(False)

In [10]:
# load gemma model 
model = HookedSAETransformer.from_pretrained("gemma-2-2b", device = device)

# # load sae on res stream of gemma model, plus cfg and sparsity val
# sae, cfg_dict, sparsity = SAE.from_pretrained(
#     release = "gemma-scope-2b-pt-res",
#     sae_id = "layer_14/width_16k/average_l0_83",
#     device = device
# )

num_layer = 10
sae, cfg_dict, sparsity = SAE.from_pretrained(
    release = "gemma-scope-2b-pt-res",
    sae_id = f"layer_{num_layer}/width_16k/average_l0_39",
    device = device
)
# get layers from here: https://jbloomaus.github.io/SAELens/sae_table/#gemma-scope-2b-pt-res

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loaded pretrained model gemma-2-2b into HookedTransformer


In [11]:
from datasets import load_dataset
from transformers import AutoTokenizer
from torch.utils.data import DataLoader
from collections import defaultdict
import torch
from transformer_lens.utils import tokenize_and_concatenate

# Load datasets
negative_dataset = load_dataset('csv', data_files='dataset/spanish_harmful.csv')['train']
positive_dataset = load_dataset('json', data_files='dataset/alpaca_spanish.json')['train']

# Rename columns to 'text'
negative_dataset = negative_dataset.rename_column('target', 'text')
positive_dataset = positive_dataset.rename_column('output', 'text')

# Select subsets
NEG_SET_SIZE = 200
POS_SET_SIZE = 200

negative_dataset = negative_dataset.select(range(NEG_SET_SIZE))
positive_dataset = positive_dataset.select(range(POS_SET_SIZE))
# # print length of datasets
# print(len(negative_dataset))
# print(len(positive_dataset))

# Load tokenizer
pos_token_dataset = tokenize_and_concatenate(
    dataset=positive_dataset,  # type: ignore
    tokenizer=model.tokenizer,  # type: ignore
    streaming=True,
    max_length=sae.cfg.context_size,
    add_bos_token=sae.cfg.prepend_bos,
)

neg_token_dataset = tokenize_and_concatenate(
    dataset=negative_dataset,  # type: ignore
    tokenizer=model.tokenizer,  # type: ignore
    streaming=True,
    max_length=sae.cfg.context_size,
    add_bos_token=sae.cfg.prepend_bos,
)


200
200


Map:   0%|          | 0/200 [00:00<?, ? examples/s]